In [1]:
#INTSALL DEPENDENCIES:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe scikit-learn matplotlib

In [26]:
#IMPORT DEPENDENCIES AND LIBRARIES
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [27]:
#MP Holistic makes it easier to draw and map the keypoints of the webcam feed
mp_holistic = mp.solutions.holistic # Holistic model - to make detections
mp_drawing = mp.solutions.drawing_utils # Drawing utilities - to draw them

In [28]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Converts the colour of the image: BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [29]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand 

In [31]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(30,144,255), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [32]:
def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    #Concatenate all the pose,left hand, right hand and face numpyarrays into a single numpy array
    return np.concatenate([pose,face,lh,rh])


In [33]:
#SETUP FOLDERS FOR COLLECTION 

#Path for exported data
DATA_PATH = os.path.join('MP_Data') 

#Actions that we try to detect 
actions = np.array(['eye', 'have', 'no', 'pain', 'yes'])

#30 Videos worth of data for each action
no_sequences = 5
#Each Video is going to be of 30 frames in length 
sequence_length = 20


In [34]:
for action in actions: #loops through the different actions 
    for sequence in range(no_sequences): #loops through the 30 different frame sets
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence))) 
            #to create the sub-directories-first creates a main folder called DATA_PATH, then a sub-folder with the name of the corresponding
            #'action' from action list, followed by creation of sequence folders [ 0....29] 
        except:
            #if folders are already created then we simply pass that iteration of the loop - error handlin
            pass

In [15]:
# Path to the folder containing the images
input_data_path = "input_data"
# Parameters: makes an intial detection and tracks the keypoints starting from there
with mp_holistic.Holistic(static_image_mode=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 
    for action in actions:
        action_folder = os.path.join(input_data_path, action)
        # Check if the current action folder exists
        if os.path.isdir(action_folder):
            for sequence in range(no_sequences):
                sequence_folder = os.path.join(action_folder, str(sequence))
                if os.path.isdir(sequence_folder):
                    for frame_num, image_file in enumerate(os.listdir(sequence_folder)):
                        if frame_num < sequence_length:
                            image_path = os.path.join(sequence_folder, image_file)
                            # Read the image
                            image = cv2.imread(image_path)
                            # Perform landmark detection
                            image, results = mediapipe_detection(image, holistic)
                            print(results)
                            #Draw landmarks 
                            draw_styled_landmarks(image, results)
                            # Extract keypoints
                            keypoints = extract_keypoints(results)
                            # Save the keypoints as a numpy array
                            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                            np.save(npy_path, keypoints)
                            
                            # Press 'q' to break the loop if needed
                            if cv2.waitKey(10) & 0xFF == ord('q'):
                                break
                    
    # Close OpenCV windows
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [35]:
#PRE-processing Data , Create Labels and Features
#Import keras utilities

from sklearn.model_selection import train_test_split # allows us to spilt/partition data into testing and training portion
from tensorflow.keras.utils import to_categorical #to convert data into encoded data

In [36]:
#Create a label array or dictionary - to reperesent each of the different actions 
label_map = {label:num for num, label in enumerate(actions)}

In [37]:
label_map

{'eye': 0, 'have': 1, 'no': 2, 'pain': 3, 'yes': 4}

In [38]:
#Create one big array: compressing data of all three actions , 30 arrays for each action * 3 
#Each frame collects 1662 landmarks arrays data 

#Create two arrays sequences (represents x or feature data) - to train the model , 
#labels (represent y data) :
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = [] #represent the 30 frames for each video/sequence
        for frame_num in range(sequence_length): #loop through the 30 frames 
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))) 
            #grabs the corresponding frame data from the specified memory path and file (sequence number)
            #append the extracted data for each frame to windows array
            window.append(res)
        #append the information of all the 30 frames for one video for each action collected in window temp array to sequences array
        sequences.append(window)
        labels.append(label_map[action])

In [39]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
#Converts to binary flags/encoded data

In [40]:
#Perform a training and testing partition using 'train test split function'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
#Unpacking the results of the train_test_split 

In [41]:
X_train.shape

(23, 20, 1662)

In [42]:
#BUILD AND TRAIN LSTM Neural Network
#Import the sequential model, LSTM layer and dense layer
from tensorflow.keras.models import Sequential         #allows us to build sequential neural network
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard     #helps to trace and monitor model while training 

In [43]:
#TensorBoard is the web app that is offered as a part of tensor flow package 
#Monitor the accuracy and training of the model
#Visualise the logs of training 

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [44]:
model = Sequential() 
# Instantiating the model

model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(20,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu')) #returns false as next layer will be a dense layer
# Adding three sets of LSTM layers 
#64 LSTM units
#return_sequences=True : If you are stacking the layer, you need to return the sequences, for the next layer 

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))  
#activation = softmax // returns those values which are within the probability range 0 to sum of all values
#Provides an array of probabilities add up to one : highest value of the array- highest probability index position corresponds to action detetcted 
#Dense Layers


In [45]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#Multi-Class Classification Model, hence we use loss='categorical_crossentropy' 
#metrics=['categorical_accuracy'] to track the accuracy of the model as we train it


In [46]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])
#The number of epochs is a hyperparameter in machine learning and deep learning that determines how many times the entire training dataset is 
#passed forward and backward through the neural network or machine learning model. 
#Each pass through the entire dataset is called one epoch.

Epoch 1/2000
1/1 [==============================] - 6s 6s/step - loss: 1.6240 - categorical_accuracy: 0.1739
Epoch 2/2000
1/1 [==============================] - 1s 787ms/step - loss: 1.7043 - categorical_accuracy: 0.1739
Epoch 3/2000
1/1 [==============================] - 0s 156ms/step - loss: 1.6585 - categorical_accuracy: 0.1739
Epoch 4/2000
1/1 [==============================] - 0s 126ms/step - loss: 2.0669 - categorical_accuracy: 0.1739
Epoch 5/2000
1/1 [==============================] - 0s 109ms/step - loss: 1.6436 - categorical_accuracy: 0.1739
Epoch 6/2000
1/1 [==============================] - 0s 133ms/step - loss: 1.6657 - categorical_accuracy: 0.2174
Epoch 7/2000
1/1 [==============================] - 0s 133ms/step - loss: 1.5140 - categorical_accuracy: 0.3478
Epoch 8/2000
1/1 [==============================] - 0s 124ms/step - loss: 1.6910 - categorical_accuracy: 0.2174
Epoch 9/2000
1/1 [==============================] - 0s 119ms/step - loss: 1.5838 - categorical_accuracy: 0.

KeyboardInterrupt: 

In [23]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 165       
Total params: 596,741
Trainable params: 596,741
Non-trainable params: 0
__________________________________________________

In [47]:
#Save Weights
model.save('action.h5')
model.load_weights('action.h5')

In [48]:
#Test in Real Time
colors = [(245,117,16), (117,245,16), (16,117,245),(16,117,245),(16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [49]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-20:]
        
        if len(sequence) == 20:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [25]:
cap.release()
cv2.destroyAllWindows()

IndentationError: unexpected indent (4015423721.py, line 2)